In [1]:
!pip install pyspark requests

from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, lit, col
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=298e8a31b2dd26b628f1d46aa052e31717766c63d686aea51d452b7c0abe43ce
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
Mounted at /content/drive


In [2]:
spark = SparkSession.builder.appName("Silver").getOrCreate()

In [3]:
# Leitura dos dados que foram escritos na camada bronze
bronze_file_path = '/content/drive/My Drive/bronze.json'
df_bronze = spark.read.json(bronze_file_path)

In [5]:
df_silver_drop.show()

+--------------------+--------------------+------------+--------------+--------------+
|                  id|                name|brewery_type|          city|         state|
+--------------------+--------------------+------------+--------------+--------------+
|e5f3e72a-fee2-481...|12 Acres Brewing ...|       micro|     Killeshin|         Laois|
|46839a79-b7bf-473...|        1817 Brewery|       micro|       okolona|   Mississippi|
|84bd3b3c-bd2d-4e0...|     1717 Brewing Co|       micro|    Des Moines|          Iowa|
|9f1852da-c312-42d...|10 Barrel Brewing...|       large|          Bend|        Oregon|
|ee6d39c6-092f-462...|    16 Stone Brewpub|     brewpub|Holland Patent|      New York|
|5128df48-79fc-4f0...|    (405) Brewing Co|       micro|        Norman|      Oklahoma|
|b51f3cdf-60ff-4ae...| 18th Street Brewery|       micro|          Gary|       Indiana|
|f41a0c47-ba9b-454...|1718 Ocracoke Bre...|     brewpub|      Ocracoke|North Carolina|
|0faa0fb2-fffa-416...|12 West Brewing C...|

In [6]:
# Retirando as duplicatas e escolhendo as colunas que serão relevantes
df_silver = df_bronze.dropDuplicates()
df_silver_drop = df_silver.select("id", "name", "brewery_type", "city", "state")

In [9]:
# Renomeando as colunas para que a primeira letra seja maiúscula, apenas para uma melhor nomenclatura
df_silver_rename = df_silver_drop.withColumnRenamed("id", "Id") \
    .withColumnRenamed("name", "Name") \
    .withColumnRenamed("brewery_type", "Brewery_type") \
    .withColumnRenamed("city", "City") \
    .withColumnRenamed("state", "State") \

df_silver_rename.show(50)

+--------------------+--------------------+------------+----------------+--------------+
|                  Id|                Name|Brewery_type|            City|         State|
+--------------------+--------------------+------------+----------------+--------------+
|e5f3e72a-fee2-481...|12 Acres Brewing ...|       micro|       Killeshin|         Laois|
|46839a79-b7bf-473...|        1817 Brewery|       micro|         okolona|   Mississippi|
|84bd3b3c-bd2d-4e0...|     1717 Brewing Co|       micro|      Des Moines|          Iowa|
|9f1852da-c312-42d...|10 Barrel Brewing...|       large|            Bend|        Oregon|
|ee6d39c6-092f-462...|    16 Stone Brewpub|     brewpub|  Holland Patent|      New York|
|5128df48-79fc-4f0...|    (405) Brewing Co|       micro|          Norman|      Oklahoma|
|b51f3cdf-60ff-4ae...| 18th Street Brewery|       micro|            Gary|       Indiana|
|f41a0c47-ba9b-454...|1718 Ocracoke Bre...|     brewpub|        Ocracoke|North Carolina|
|0faa0fb2-fffa-416...

In [10]:
# Criação de uma nova coluna concatenando os dados de duas para trazer a localização das cervejarias
df_silver_partition = df_silver_rename.withColumn("Brewery_Location", concat(col("City"), lit(", "), col("State")))
df_silver_partition.show(50)

+--------------------+--------------------+------------+----------------+--------------+--------------------+
|                  Id|                Name|Brewery_type|            City|         State|    Brewery_Location|
+--------------------+--------------------+------------+----------------+--------------+--------------------+
|e5f3e72a-fee2-481...|12 Acres Brewing ...|       micro|       Killeshin|         Laois|    Killeshin, Laois|
|46839a79-b7bf-473...|        1817 Brewery|       micro|         okolona|   Mississippi|okolona, Mississippi|
|84bd3b3c-bd2d-4e0...|     1717 Brewing Co|       micro|      Des Moines|          Iowa|    Des Moines, Iowa|
|9f1852da-c312-42d...|10 Barrel Brewing...|       large|            Bend|        Oregon|        Bend, Oregon|
|ee6d39c6-092f-462...|    16 Stone Brewpub|     brewpub|  Holland Patent|      New York|Holland Patent, N...|
|5128df48-79fc-4f0...|    (405) Brewing Co|       micro|          Norman|      Oklahoma|    Norman, Oklahoma|
|b51f3cdf-

In [12]:
# Retirando os dados nulos, substituindo por espaços vazios
df_silver_final = df_silver_partition.fillna({'Id': '', 'Name': '', 'Brewery_type': '', 'City': '', 'State': ''})
df_silver_final.show(50)

+--------------------+--------------------+------------+----------------+--------------+--------------------+
|                  Id|                Name|Brewery_type|            City|         State|    Brewery_Location|
+--------------------+--------------------+------------+----------------+--------------+--------------------+
|e5f3e72a-fee2-481...|12 Acres Brewing ...|       micro|       Killeshin|         Laois|    Killeshin, Laois|
|46839a79-b7bf-473...|        1817 Brewery|       micro|         okolona|   Mississippi|okolona, Mississippi|
|84bd3b3c-bd2d-4e0...|     1717 Brewing Co|       micro|      Des Moines|          Iowa|    Des Moines, Iowa|
|9f1852da-c312-42d...|10 Barrel Brewing...|       large|            Bend|        Oregon|        Bend, Oregon|
|ee6d39c6-092f-462...|    16 Stone Brewpub|     brewpub|  Holland Patent|      New York|Holland Patent, N...|
|5128df48-79fc-4f0...|    (405) Brewing Co|       micro|          Norman|      Oklahoma|    Norman, Oklahoma|
|b51f3cdf-

In [13]:
# Escrita no drive em formato parquet
df_silver_final.write.mode("overwrite").partitionBy("Brewery_Location").parquet("/content/drive/My Drive/silver.parquet")